<a href="https://colab.research.google.com/github/airesx2/Internship-2025/blob/main/circuit_injection_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install cudaq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.3/202.3 MB 6.5 MB/s eta 0:00:00
  Created wheel for cudaq: filename=cudaq-0.11.0-py3-none-any.whl size=6897 sha256=2e112d7137ca5a36b5544a1fc653882d518c725a3c46a539b5ddc6c818862ac0
  Stored in directory: /root/.cache/pip/wheels/ad/0f/19/60352710d764963c7c1cb5d5f1ab78dc76e1b542899b33c35a
Successfully built cudaq


Safe Grover's Algorithm VS. Malicious Oracle Injection

In [2]:
import cudaq
import math

In [3]:
#cudaq simulation of a circuit injection attack (stealth/sabotage/silent)

#Safe Grover's Algorithm
target = [1,0,1]
n = len(target)
iter = round((math.pi / 4) * math.sqrt(2 ** n))

@cudaq.kernel
def search(target: list[int], n: int, iter: int):

  #initialize qubits
  qubits = cudaq.qvector(n)

  #uniform superposition
  h(qubits)

  #oracle + diffuser loop
  for i in range(iter):

    #oracle
    for i in range(n):
      if target[i] == 0:
        x(qubits[i])
    cz([qubits[0], qubits[1]], qubits[2])
    for i in range(n):
      if target[i] == 0:
        x(qubits[i])

    #diffusion
    for q in qubits:
      h(q)
      x(q)
    cz([qubits[0], qubits[1]], qubits[2])
    for q in qubits:
      x(q)
      h(q)

    #measure
    mz(qubits)

#Injected Oracle

#STEALTH MODE
#wrong oracle but looks legit... outputs peak at 000
@cudaq.kernel #remember to kernel
def stealth(target: list[int], n: int, iter:int):
  #initialize
  qubits = cudaq.qvector(n)

  #superposition
  h(qubits)

  #malicious payload
  #ignores the target (b/c not checking for bit_pattern)
  #flips the target qubit only if all the control qubits are 1
  #always marks the 000 state
  for i in range(iter):
    x(qubits)
    cz([qubits[0], qubits[1]], qubits[2])
    x(qubits)

    #diffusion
    for q in qubits:
      h(q)
      x(q)
    cz([qubits[0], qubits[1]], qubits[2])
    for q in qubits:
      x(q)
      h(q)

  #measurement
  mz(qubits)

#SABOTAGE MODE
#no amplifying/broken gates... outputs flat distri/noise
@cudaq.kernel
def sabotage(target: list[int], n: int, iter:int):
  #initialize
  qubits = cudaq.qvector(n)

  #superposition
  h(qubits)

  #broken oracle
  for i in range(iter):
    x(qubits)
    cz([qubits[0], qubits[1]], qubits[2])
    x(qubits)

    #diffusion
    x(qubits[0]) #add incorrect noise

  #measurement
  mz(qubits)

#SILENT MODE
#does nothing and breaks in silence... outputs flat noise/ random distr
@cudaq.kernel
def silent(target: list[int], n: int, iter:int):
  qubits = cudaq.qvector(n)

  h(qubits)

  #fake gates that do absolutely nothing but looks like something
  #no oracle diffuser action at all
  for i in range(iter):
    #"Oracle"
    h(qubits[i % n]) #useless

    #"Diffuser"
    x(qubits[i % n]) #useless

    h(qubits[0]) #useless

  mz(qubits) #measure


#compare safe grover's alg & malicious injected oracle
mode = input("Select a mode [stealth / sabotage / silent]... ")
print("\n")
print("Safe Grover's Algorithm...")
print(cudaq.sample(search, target, n, iter))
print(f"Malicious Oracle Injection({mode})...")
if mode == "stealth":
  print(cudaq.sample(stealth, target, n, iter))
elif mode == "sabotage":
  print(cudaq.sample(sabotage, target, n, iter))
elif mode == "silent":
  print(cudaq.sample(silent, target, n, iter))


Select a mode [stealth / sabotage / silent]... silent


Safe Grover's Algorithm...
{ 000:6 001:6 010:10 011:12 100:7 101:948 110:7 111:4 }

Malicious Oracle Injection(silent)...
{ 110:493 111:507 }



Runtime Hijacking Demo


In [1]:
import cudaq
import math

In [6]:
#Simulate runtime hijacking attach
#Added a simple tamper check

#Safe Grover's Algorithm
target = [1,0,1]
n = len(target)
iter = round((math.pi / 4) * math.sqrt(2 ** n))

@cudaq.kernel
def search(target: list[int], n: int, iter: int):

  #initialize qubits
  qubits = cudaq.qvector(n)

  #uniform superposition
  h(qubits)

  #oracle + diffuser loop
  for i in range(iter):

    #oracle
    for i in range(n):
      if target[i] == 0:
        x(qubits[i])
    cz([qubits[0], qubits[1]], qubits[2])
    for i in range(n):
      if target[i] == 0:
        x(qubits[i])

    #diffusion
    for q in qubits:
      h(q)
      x(q)
    cz([qubits[0], qubits[1]], qubits[2])
    for q in qubits:
      x(q)
      h(q)

    #measure
    mz(qubits)

#Injected Oracle
#STEALTH MODE
@cudaq.kernel #remember to kernel
def stealth(target: list[int], n: int, iter:int):
  #initialize
  qubits = cudaq.qvector(n)

  #superposition
  h(qubits)

  #malicious payload
  #ignores the target (b/c not checking for bit_pattern)
  #flips the target qubit only if all the control qubits are 1
  #this specific incorrect oracle always marks the 000 state
  for i in range(iter):
    x(qubits)
    cz([qubits[0], qubits[1]], qubits[2])
    x(qubits)

    #diffusion
    #wrong oracle but looks legit... outputs peak at 000
    for q in qubits:
      h(q)
      x(q)
    cz([qubits[0], qubits[1]], qubits[2])
    for q in qubits:
      x(q)
      h(q)

  #measurement
  mz(qubits)

#dispatcher function
def run_alg():
  result = cudaq.sample(search, target, n, iter)
  print(f"Search results: {result}")
og_kernel = search
globals()['search'] = stealth #swaps kernel reference at runtime

#basic tamper check / auto revert
def defender(kernel):
  if kernel is not og_kernel:
    print("DEFENDER ALERT! Kernel tampering detected.")
    print("Reverting... ")
    globals()['search'] = og_kernel
  else:
    print("Kernel integrity intact.")


#run
usr = input("Start running algorithm [y/n]? ")
if usr == "y":
  check = input("Activate defender check [y/n]? ")
  print("\n")
  if(check == "y"):
    defender(search)
  else:
    print("Defender check was not activated.")
  run_alg()
  print("Runtime Hijacking Complete.")
else:
  print("Aborted.")
  exit()


Start running algorithm [y/n]? y
Activate defender check [y/n]? n


Defender check was not activated.
Search results: { 000:941 001:8 010:10 011:6 100:14 101:5 110:7 111:9 }

Runtime Hijacking Complete.
